In [ ]:
import importlib.util
import sys
import torch
import math
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from datasets import load_dataset, Dataset, concatenate_datasets
from sklearn.model_selection import train_test_split
import psutil
import gc
import time
import requests

# Check if rouge_score is installed
if importlib.util.find_spec("rouge_score") is None:
    print("Error: 'rouge_score' module not found. Please install it manually in a virtual environment or using pipx:")
    print("1. Create a virtual environment: python3 -m venv venv")
    print("2. Activate it: source venv/bin/activate")
    print("3. Install: pip install rouge_score")
    print("Or use pipx: pipx install rouge_score")
    sys.exit(1)

from rouge_score import rouge_scorer
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification, BertTokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling, pipeline

# Load and merge datasets
def load_and_merge_datasets():
    # STEP 1: Load HOPE Therapy Data
    hope_path = "/content/SPARTA_WSDM2022/HOPE_data/HOPE_therapy_session_transcripts"
    try:
        files = [f for f in os.listdir(hope_path) if f.endswith(".csv")]
    except FileNotFoundError:
        print(f"Error: HOPE dataset directory not found at {hope_path}")
        files = []

    hope_pairs = []
    for file in files:
        df = pd.read_csv(os.path.join(hope_path, file))
        # Map the Type column to proper roles
        df['Speaker'] = df['Type'].map({'T': 'Therapist', 'P': 'Client'})
        df['Content'] = df['Utterance']  # Rename for consistency

        # Extract consecutive client-therapist exchanges
        for i in range(1, len(df)):
            if df.loc[i-1, 'Speaker'] == "Client" and df.loc[i, 'Speaker'] == "Therapist":
                hope_pairs.append({
                    "prompt": f"Client: {df.loc[i-1, 'Content']}",
                    "response": f"Therapist: {df.loc[i, 'Content']}",
                    "source": "HOPE"
                })

    print(f"Extracted {len(hope_pairs)} dialogue pairs from HOPE dataset")

    # STEP 2: Load EmpatheticDialogues
    try:
        empathetic_ds = load_dataset("empathetic_dialogues")
        empathy_pairs = []

        # Process conversations to get contextual exchanges
        prev_conv_id = None
        context = ""

        for row in empathetic_ds['train']:
            if row['utterance_idx'] > 0 and row['conv_id'] == prev_conv_id:
                empathy_pairs.append({
                    "prompt": f"Client: {context}",
                    "response": f"Therapist: {row['utterance']}",
                    "emotion": row['context'],
                    "source": "EmpatheticDialogues"
                })

            context = row['utterance']
            prev_conv_id = row['conv_id']

        print(f"Extracted {len(empathy_pairs)} dialogue pairs from EmpatheticDialogues dataset")
    except Exception as e:
        print(f"Error loading EmpatheticDialogues: {e}")
        empathy_pairs = []

    # STEP 3: Load CounselChat
    try:
        # Download CounselChat data
        url = "https://raw.githubusercontent.com/nbertagnolli/counsel-chat/master/data/counselchat-data.csv"
        response = requests.get(url)
        with open("counselchat-data.csv", "wb") as f:
            f.write(response.content)

        cc_df = pd.read_csv("counselchat-data.csv")
        counsel_pairs = []

        for _, row in cc_df.iterrows():
            if pd.notnull(row['questionText']) and pd.notnull(row['answerText']):
                counsel_pairs.append({
                    "prompt": f"Client: {row['questionText']}",
                    "response": f"Therapist: {row['answerText'].replace('<p>','')}",
                    "source": "CounselChat"
                })

        print(f"Extracted {len(counsel_pairs)} dialogue pairs from CounselChat dataset")
    except Exception as e:
        print(f"Error loading CounselChat: {e}")
        counsel_pairs = []

    # STEP 4: Merge All Dialogues
    dialogue_data = hope_pairs + empathy_pairs + counsel_pairs
    print(f"Total dialogue pairs: {len(dialogue_data)}")

    # Optional: Sample checking of data quality
    print("\nSample data from each source:")
    for source in ["HOPE", "EmpatheticDialogues", "CounselChat"]:
        samples = [d for d in dialogue_data if d.get("source") == source]
        if samples:
            print(f"\n{source} sample:")
            sample = np.random.choice(samples)
            print(f"Prompt: {sample['prompt']}")
            print(f"Response: {sample['response']}")

    # Convert to Hugging Face Dataset (excluding 'source' and 'emotion' for consistency)
    dialogue_data = [{'prompt': d['prompt'], 'response': d['response']} for d in dialogue_data]
    return Dataset.from_list(dialogue_data)

# Load and split dataset
dialogue_data = load_and_merge_datasets()
train_data, val_data = train_test_split(dialogue_data.to_pandas(), test_size=0.2, random_state=42)
train_dataset = Dataset.from_pandas(train_data.reset_index(drop=True))
val_dataset = Dataset.from_pandas(val_data.reset_index(drop=True))

# Define MentalHealthChatbot class
class MentalHealthChatbot:
    def __init__(self, bert_model, bert_tokenizer, gpt_model, gpt_tokenizer):
        self.bert_model = bert_model
        self.bert_tokenizer = bert_tokenizer
        self.gpt_model = gpt_model
        self.gpt_tokenizer = gpt_tokenizer
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Move models to correct device
        self.bert_model.to(self.device)
        self.gpt_model.to(self.device)

        # Set models to evaluation mode
        self.bert_model.eval()
        self.gpt_model.eval()

    def analyze_emotion(self, text):
        """Use BERT to analyze the emotional content and context of user input"""
        inputs = self.bert_tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = self.bert_model(**inputs)
            emotion_scores = outputs.logits[0]
            emotion_id = torch.argmax(emotion_scores).item()
            emotion = self.bert_model.config.id2label.get(emotion_id, "neutral")
            confidence = torch.softmax(emotion_scores, dim=0)[emotion_id].item()

        return emotion, confidence

    def generate_response(self, user_input, max_length=150):
        """Generate therapeutic response using BERT emotion analysis and GPT-2"""
        emotion, confidence = self.analyze_emotion(user_input)
        enhanced_prompt = f"Client (feeling {emotion}): {user_input}\nTherapist:"
        inputs = self.gpt_tokenizer(enhanced_prompt, return_tensors="pt")
        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        output_sequences = self.gpt_model.generate(
            **inputs,
            max_length=len(inputs["input_ids"][0]) + max_length,
            num_return_sequences=1,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=self.gpt_tokenizer.eos_token_id
        )

        full_text = self.gpt_tokenizer.decode(output_sequences[0], skip_special_tokens=True)
        try:
            therapist_response = full_text.split("Therapist:", 1)[1].strip()
        except IndexError:
            therapist_response = full_text

        response_data = {
            "response": therapist_response,
            "emotion_detected": emotion,
            "confidence": confidence
        }
        return response_data

# Load models (using pre-trained for demonstration; replace with fine-tuned models after training)
try:
    bert_model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")
    bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    gpt_model = AutoModelForCausalLM.from_pretrained("gpt2")
    gpt_tokenizer = AutoTokenizer.from_pretrained("gpt2")
    gpt_tokenizer.pad_token = gpt_tokenizer.eos_token
except FileNotFoundError:
    print("Fine-tuned models not found. Using pre-trained models. Please train models for better results.")
    bert_model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")
    bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    gpt_model = AutoModelForCausalLM.from_pretrained("gpt2")
    gpt_tokenizer = AutoTokenizer.from_pretrained("gpt2")
    gpt_tokenizer.pad_token = gpt_tokenizer.eos_token

# Initialize chatbot
chatbot = MentalHealthChatbot(bert_model, bert_tokenizer, gpt_model, gpt_tokenizer)

# Test response generation
sample_input = "I've been feeling really down lately and nothing seems to help."
response = chatbot.generate_response(sample_input)
print(f"Sample Input: {sample_input}")
print(f"Detected emotion: {response['emotion_detected']} (confidence: {response['confidence']:.2f})")
print(f"Response: {response['response']}")

# Evaluation functions
def calculate_perplexity(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
    return math.exp(loss.item())

def calculate_bleu(reference, hypothesis):
    smoothie = SmoothingFunction().method4
    return sentence_bleu([reference.split()], hypothesis.split(), smoothing_function=smoothie)

def calculate_rouge(reference, hypothesis):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    return scorer.score(reference, hypothesis)

def distinct_n_gram(responses, n=2):
    all_ngrams = []
    for response in responses:
        tokens = response.split()
        ngrams = list(zip(*[tokens[i:] for i in range(n)]))
        all_ngrams.extend(ngrams)
    total = len(all_ngrams)
    unique = len(set(all_ngrams))
    return unique / total if total > 0 else 0

def measure_inference_time(model, tokenizer, prompt):
    start = time.time()
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    _ = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)
    end = time.time()
    return end - start

def get_memory_usage():
    process = psutil.Process(os.getpid())
    cpu_mem = process.memory_info().rss / 1024 ** 2  # in MB
    gpu_mem = torch.cuda.max_memory_allocated() / 1024 ** 2 if torch.cuda.is_available() else None
    torch.cuda.reset_max_memory_allocated() if torch.cuda.is_available() else None
    gc.collect()
    return {'cpu_memory_MB': cpu_mem, 'gpu_memory_MB': gpu_mem}

# Evaluation using val_dataset
input_prompts = [item["prompt"] for item in val_dataset]
reference_responses = [item["response"] for item in val_dataset]
generated_responses = []

# Generate responses using chatbot
gpt_model.eval()
for prompt in tqdm(input_prompts, desc="Generating responses"):
    response = chatbot.generate_response(prompt)
    generated_responses.append(response["response"])

# Initialize accumulators
bleu_scores = []
rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
perplexities = []
inference_times = []
memory_usages = []

for ref, gen, prompt in zip(reference_responses, generated_responses, input_prompts):
    bleu_scores.append(calculate_bleu(ref, gen))
    rouge = calculate_rouge(ref, gen)
    for key in rouge_scores:
        rouge_scores[key].append(rouge[key].fmeasure)
    perplexities.append(calculate_perplexity(gpt_model, gpt_tokenizer, gen))
    inference_times.append(measure_inference_time(gpt_model, gpt_tokenizer, prompt))
    memory_usages.append(get_memory_usage())

# Compute distinct-n
distinct_1 = distinct_n_gram(generated_responses, n=1)
distinct_2 = distinct_n_gram(generated_responses, n=2)

# Print average results
print("\n--- Evaluation Metrics ---")
print(f"Avg BLEU Score: {sum(bleu_scores)/len(bleu_scores):.4f}")
print(f"Avg ROUGE-1: {sum(rouge_scores['rouge1'])/len(rouge_scores['rouge1']):.4f}")
print(f"Avg ROUGE-2: {sum(rouge_scores['rouge2'])/len(rouge_scores['rouge2']):.4f}")
print(f"Avg ROUGE-L: {sum(rouge_scores['rougeL'])/len(rouge_scores['rougeL']):.4f}")
print(f"Avg Perplexity: {sum(perplexities)/len(perplexities):.4f}")
print(f"Distinct-1: {distinct_1:.4f}")
print(f"Distinct-2: {distinct_2:.4f}")
print(f"Avg Inference Time: {sum(inference_times)/len(inference_times):.4f} seconds")
print(f"Avg CPU Memory: {sum([m['cpu_memory_MB'] for m in memory_usages])/len(memory_usages):.2f} MB")
if torch.cuda.is_available():
    print(f"Avg GPU Memory: {sum([m['gpu_memory_MB'] for m in memory_usages if m['gpu_memory_MB'] is not None])/len(memory_usages):.2f} MB")